In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Set up the credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('lithuanian-427002-52bf25ae04cd.json', scope)
client = gspread.authorize(creds)

In [47]:
# Open the Google Sheet
sheet_url = 'https://docs.google.com/spreadsheets/d/1chGs5Aj4rS38_R6Fl7B5a5sfH8MWvvuYCI1peioGW64/edit#gid=786163238'
sheet = client.open_by_url(sheet_url)

In [48]:
# Get the list of sheet names
sheet_names = [ws.title for ws in sheet.worksheets()]

# Create a dictionary to store the DataFrames
dataframes = {}

# Iterate over each sheet and create a DataFrame
for sheet_name in sheet_names:
    worksheet = sheet.worksheet(sheet_name)
    data = worksheet.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    dataframes[sheet_name] = df

# Print the sheet names and DataFrames
for sheet_name, df in dataframes.items():
    print(f"Sheet Name: {sheet_name}")
    # print(df)
    # print()

Sheet Name: nouns
Sheet Name: sentence-templates
Sheet Name: adjectives
Sheet Name: pronouns
Sheet Name: verb-information
Sheet Name: verb-conjugation
Sheet Name: numbers


In [49]:
sheet_names

['nouns',
 'sentence-templates',
 'adjectives',
 'pronouns',
 'verb-information',
 'verb-conjugation',
 'numbers']

In [50]:
worksheet = sheet.worksheet('nouns')

In [51]:
data = worksheet.get_all_values()

In [52]:
dataframes['nouns']

,nominative,plural,countable,gender,genitive singular,genitive plural,accusative singular,accusative plural,instrumental singular,instrumental plural,dative singular,dative plural,declension,category
0,tėvas,tėvai,countable,masculine,,,,,,,tėvui,tėvams,I,person
1,brolis,broliai,countable,masculine,,,,,,,broliui,broliams,I,person
2,pabrolys,pabroliai,countable,masculine,,,,,,,pabroliui,pabroliams,I,person
3,mama,mamos,countable,feminine,,,,,,,mamai,mamoms,II,person
4,dukra,dukros,countable,feminine,,,,,,,dukrai,dukroms,II,person
5,sesė,sesės,countable,feminine,,,,,,,sesei,sesėms,II,person
6,akis,akys,countable,feminine,,,,,,,akiai,akims,III,thing
7,dantis,dantys,countable,masculine,,,,,,,dančiui,dantims,III,thing
8,sūnus,sūnūs,countable,masculine,,,,,,,sūnui,sūnums,IV,person
9,profesorius,profesoriai,countable,masculine,,,,,,,profesoriui,profesoriams,IV,person


In [53]:
dataframes['verb-conjugation']

,index,subject,tense,conjugation,infinitive
0,0,aš,present_tense,matau,matyti
1,1,tu,present_tense,matai,matyti
2,2,jis,present_tense,mato,matyti
3,3,ji,present_tense,mato,matyti
4,4,mes,present_tense,matome,matyti
...,...,...,...,...,...
247,37,ji,past_freq_tense,valgydavo,valgyti
248,38,mes,past_freq_tense,valgydavome,valgyti
249,39,jūs,past_freq_tense,valgydavote,valgyti
250,40,jie,past_freq_tense,valgydavo,valgyti


In [54]:
infinitives = dataframes['verb-conjugation']['infinitive'].unique()

In [55]:
infinitives

array(['matyti', 'būti', 'mėgti', 'norėti', 'prašyti', 'valgyti'],
      dtype=object)

In [56]:
dataframes['pronouns']

,nominative,possessive,dative,genitive,accusative,instrumental,locative
0,aš,mano,man,manęs,mane,manimi,manyje
1,tu,tavo,tau,tavęs,tave,tavimi,tavyje
2,jis,jo,jam,jo,jį,juo,jame
3,ji,jos,jai,jos,ją,ja,joje
4,mes,mūsų,mums,mūsų,mus,mumis,mumyse
5,jūs,jūsų,jums,jūsų,jus,jumis,jumyse
6,jie,jų,jiems,jų,juos,jais,juose
7,jos,jų,joms,jų,jas,jomis,jose


In [57]:
import random

def generate_sentence(nouns, pronouns, template="{pronoun-dative} patinka {noun-food-countable-nominative}."):
    # Filter countable food nouns
    food_nouns = nouns[(nouns['category'] == 'food') & (nouns['countable'] == 'countable')]
    
    # Select a random pronoun and food noun
    pronoun = pronouns.loc[random.choice(pronouns.index)]
    noun = food_nouns.loc[random.choice(food_nouns.index)]
    
    # Generate the prompt
    prompt = f"{pronoun['nominative']}, patinka, {noun['nominative']}"
    
    # Generate the answer
    # answer = template.format(pronoun=pronoun['dative'], noun=noun['nominative'])
    answer = f"{pronoun['dative'].title()} patinka {noun['nominative']}."
    # Get the category (declension) of the noun
    category = noun['declension']
    
    return prompt, answer, category

In [58]:
nouns = dataframes['nouns']

In [59]:
pronouns = dataframes['pronouns']

In [74]:
generate_sentence(nouns, pronouns, template="{pronoun['dative']} patinka {noun['nominative']}.")

('ji, patinka, šokoladas', 'Jai patinka šokoladas.', 'I')

In [61]:
pronouns

,nominative,possessive,dative,genitive,accusative,instrumental,locative
0,aš,mano,man,manęs,mane,manimi,manyje
1,tu,tavo,tau,tavęs,tave,tavimi,tavyje
2,jis,jo,jam,jo,jį,juo,jame
3,ji,jos,jai,jos,ją,ja,joje
4,mes,mūsų,mums,mūsų,mus,mumis,mumyse
5,jūs,jūsų,jums,jūsų,jus,jumis,jumyse
6,jie,jų,jiems,jų,juos,jais,juose
7,jos,jų,joms,jų,jas,jomis,jose


In [62]:
food_nouns = nouns[(nouns['category'] == 'food') & (nouns['countable'] == 'countable')]
food_nouns

,nominative,plural,countable,gender,genitive singular,genitive plural,accusative singular,accusative plural,instrumental singular,instrumental plural,dative singular,dative plural,declension,category
13,žuvis,žuvys,countable,feminine,žuvies,žuvų,žuvį,žuvis,žuvimi,žuvimis,žuviai,žuvims,III,food
15,kumpis,kumpiai,countable,masculine,kumpio,kumpių,kumpį,kumpius,kumpiu,kumpiais,kumpiui,kumpiams,III,food
16,dešrelė,dešrelės,countable,feminine,dešrelės,dešrelių,dešrelę,dešreles,dešrele,dešrelėmis,dešrelei,dešrelėms,III,food
17,dešra,dešros,countable,feminine,dešros,dešrų,dešrą,dešras,dešra,dešromis,dešrai,dešroms,I,food
20,sūris,sūriai,countable,masculine,sūrio,sūrių,sūrį,sūrius,sūriu,sūriais,sūriui,sūriams,III,food
23,karpis,karpiai,countable,masculine,karpio,karpių,karpį,karpius,karpiu,karpiais,karpiui,karpiams,III,food
24,duona,duonos,countable,feminine,duonos,duonų,duoną,duonas,duona,duonomis,duonai,duonoms,I,food
25,bandelė,bandelės,countable,feminine,bandelės,bandelių,bandelę,bandeles,bandele,bandelėmis,bandelei,bandelėms,III,food
26,riestainis,riestainiai,countable,masculine,riestainio,riestainų,riestainį,riestainius,riestainiu,riestainiu,riestainiui,riestainiams,III,food
27,batonas,batonai,countable,masculine,batono,batonų,batoną,batonus,batonu,batonais,batonui,batonams,I,food


In [63]:
# Select a random pronoun and food noun
pronoun = pronouns.loc[random.choice(pronouns.index)]
noun = food_nouns.loc[random.choice(food_nouns.index)]

# Generate the prompt
prompt = f"{pronoun['nominative']}, patinka, {noun['nominative']}"

In [64]:
prompt

'tu, patinka, agurkas'

In [65]:
pronoun['dative']

'tau'

In [66]:
noun['nominative']

'agurkas'

In [67]:
f"{pronoun['dative'].title()} patinka {noun['nominative']}."

'Tau patinka agurkas.'